# Probabilistic Robotics for State Estimation

Probabilistic robotics is a framework that deals with uncertainty in robot perception and action by using probability theory. It's especially valuable for state estimation - determining where a robot is and what's around it in uncertain environments.

## Core Concept

The fundamental idea is that a robot's belief about its state (position, orientation, etc.) and the environment is represented as a probability distribution rather than as single, deterministic values. This approach acknowledges the inherent uncertainty in:

1. **Sensor measurements** (noise, limited range, occlusions)
2. **Motion actions** (wheel slippage, uneven terrain)
3. **Environment dynamics** (moving objects, changing conditions)



## Mathematical Framework

The mathematical foundation lies in Bayesian probability theory, where we update beliefs based on new evidence.

### Bayes' Rule

$$
p(x|z) = \frac{p(z|x) \cdot p(x)}{p(z)}
$$

Where:
- $p(x)$ is the prior belief about state $x$
- $p(z|x)$ is the likelihood of getting measurement $z$ given state $x$
- $p(x|z)$ is the posterior belief after incorporating measurement $z$
- $p(z)$ is a normalizing factor



### State Estimation Process

State estimation typically follows these steps:

1. **Prediction Step (Motion Model)**: When a robot moves, uncertainty increases.
   
   $$p(x_t|u_t,x_{t-1}) $$
   
   This transition model describes how state $x$ changes when control $u$ is applied.

2. **Update Step (Measurement Model)**: When sensors provide data, uncertainty decreases.
   
   $$p(z_t|x_t)$$
   
   This sensor model describes the probability of getting measurement $z$ in state $x$.

3. **Belief Update**: Combining the above using Bayes' rule.
   
   $$bel(x_t) = \eta \cdot p(z_t|x_t) \int p(x_t|u_t,x_{t-1}) \cdot bel(x_{t-1}) \, dx_{t-1}$$
   
   Where $\eta$ is a normalizing constant and $bel(x_t)$ is the belief at time $t$.



## Common Algorithms

1. **Kalman Filter**: For linear systems with Gaussian noise.
2. **Extended Kalman Filter (EKF)**: For nonlinear systems, using linearization.
3. **Particle Filter**: For arbitrary distributions, using sampling.
4. **Histogram Filter**: For discrete state spaces.



## Simple Example: Robot Localization

Let's consider a simple 1D robot moving along a line, with:
- A motion model that attempts to move forward 1 unit each step but has noise
- A sensor that can detect a pole in the tunnel with some uncertainty

### Initial Belief
The robot starts with a uniform belief about its position - it could be anywhere with equal probability.

### Prediction Step
The robot attempts to move forward 1 unit by applying motor controls. Due to uncertainty in motion:
- If it's at position $x$ now, it might end up at $x+0.8$, $x+1.0$, or $x+1.2$ with different probabilities
- This "smears" the belief forward and increases uncertainty

Mathematically:
$$bel^-(x_t) = \int p(x_t|u_t,x_{t-1}) \cdot bel(x_{t-1}) \, dx_{t-1}$$

### Update Step
The robot takes a sensor measurement and detects a pole 2 metres away, but with some uncertainty.

If the sensor is perfect, positions exactly 2 units from the pole would have probability 1.0, and all others 0.0. But with noise:
- Positions 1.8-2.2 units from the pole might have high probability
- Positions further away have lower probability

Mathematically:
$$bel(x_t) = \eta \cdot p(z_t|x_t) \cdot bel^-(x_t)$$

### Result
After combining the prediction and update, the robot has a revised belief about its position - typically a sharper, more certain distribution than before.



## Particle Filter Implementation Example

A particle filter would represent this belief using discrete samples (particles):


This simple example demonstrates how a robot can:
1. Start with high uncertainty
2. Move and increase uncertainty further
3. Take measurements to reduce uncertainty
4. Combine these to arrive at a better state estimate

This probabilistic approach allows robots to function effectively despite the uncertainty inherent in real-world environments.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display


# Function to calculate estimated position (weighted average)
def calculate_estimated_position(particles, weights):
    return np.sum(particles * weights)

# Modify the run_simulation function to return and display the estimated position
def run_simulation(num_particles, move_distance, move_noise, measurement_noise, pole_distance, measurement):
    # Initialize particles (positions) with uniform distribution
    particles = np.random.uniform(0, 1, num_particles)  # Robot could be anywhere from 0-10
    
    # Create figure for plotting
    plt.figure(figsize=(10, 6))
    
    # Initial belief
    plt.subplot(3, 1, 1)
    plt.hist(particles, bins=100, color='b', density=True)
    plt.title("Initial Belief (Uniform between 0-10)")
    plt.xlim(0, 20)
    
    # After movement (prediction step)
    particles = particles + move_distance + np.random.normal(0, move_noise, len(particles))
    plt.subplot(3, 1, 2)
    plt.hist(particles, bins=100, density=True, color='y')
    plt.title(f"After Movement (Prediction Step): Move {move_distance} units with noise {move_noise:.2f}")
    plt.xlim(0, 20)
    
    # Calculate weights based on measurement (update step)
    expected_measurements = pole_distance - particles
    weights = np.exp(-0.5 * ((expected_measurements - measurement) / measurement_noise)**2)
    weights = weights / sum(weights)  # Normalize weights
    
    # Measures to capture the "compatibility" of our measurement given the prediction:
    # Calculate effective sample size (ESS)
    ess = 1.0 / np.sum(weights**2)
    ess_percentage = 100 * ess / num_particles
    print(f"Effective Sample Size: {ess:.1f}/{num_particles} ({ess_percentage:.1f}%)")

    # Calculate entropy of the weights
    entropy = -np.sum(weights * np.log(weights + 1e-10))  # Adding small epsilon to avoid log(0)
    entropy_normalized = entropy / np.log(num_particles)  # Normalize by maximum possible entropy
    print(f"Normalized Entropy: {entropy_normalized:.3f} (higher = more uncertain)")

    # Calculate estimated position
    estimated_pos = calculate_estimated_position(particles, weights)
    
    # Resample particles based on weights (low weight particles will be replaced by high weight particles)
    resampled_indices = np.random.choice(range(num_particles), num_particles, p=weights)
    particles = particles[resampled_indices]
    
    plt.subplot(3, 1, 3)
    plt.hist(particles, bins=100, color='g', density=True)
    plt.title(f"After Sensor Update: Est. Position = {estimated_pos:.2f}; ESS={ess_percentage:.1f}%; Entropy={entropy_normalized:.3f}")
    plt.axvline(x=estimated_pos, color='r', linestyle='--')
    plt.xlim(0, 20)
    
    plt.tight_layout()
    plt.show()

    
    return estimated_pos

# Create interactive widgets
num_particles_slider = widgets.IntSlider(value=1000, min=100, max=10000, step=100, description='Particles:')
move_distance_slider = widgets.FloatSlider(value=5.0, min=0.0, max=20.0, step=0.1, description='Move:')
move_noise_slider = widgets.FloatSlider(value=0.5, min=0.1, max=5.0, step=0.1, description='Move Noise:')
measurement_noise_slider = widgets.FloatSlider(value=0.2, min=0.001, max=5.0, step=0.1, description='Meas. Noise:')
pole_distance_slider = widgets.FloatSlider(value=8.0, min=5.0, max=20.0, step=1.0, description='Pole Pos:')
measurement_slider = widgets.FloatSlider(value=2.0, min=0.0, max=20.0, step=0.1, description='Meas. Dist:')

# Create the interactive output
interactive_output = widgets.interactive_output(run_simulation, {
    'num_particles': num_particles_slider,
    'move_distance': move_distance_slider,
    'move_noise': move_noise_slider,
    'measurement_noise': measurement_noise_slider,
    'pole_distance': pole_distance_slider,
    'measurement': measurement_slider
})

# Display widgets and output
controls = widgets.VBox([
    num_particles_slider, 
    move_distance_slider, 
    move_noise_slider, 
    measurement_noise_slider, 
    pole_distance_slider, 
    measurement_slider
])

display(widgets.HBox([controls, interactive_output]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, atan2, pi, sqrt, exp
import time
import ipywidgets as widgets
from IPython.display import display
from matplotlib.patches import Arrow
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

class DifferentialDriveRobot:
    """
    A differential drive robot model with simulated IMU and GPS sensors.
    """
    def __init__(self, x=0.0, y=0.0, theta=0.0, wheel_radius=0.1, wheel_base=0.5):
        # Robot state
        self.x = x
        self.y = y
        self.theta = theta  # Heading in radians
        
        # Robot parameters
        self.wheel_radius = wheel_radius
        self.wheel_base = wheel_base
        
        # Sensor noise parameters will be set by the simulation
        self.imu_noise_angular_velocity = 0.01  # rad/s
        self.imu_noise_linear_accel = 0.05      # m/s²
        self.gps_noise_position = 1.0           # meters
        
        # Internal state for simulation
        self.v_left = 0.0   # Left wheel velocity
        self.v_right = 0.0  # Right wheel velocity
        self.last_time = time.time()
    
    def update_wheels_velocity(self, v_left, v_right):
        """Set the wheel velocities in rad/sec."""
        self.v_left = v_left
        self.v_right = v_right
    
    def update(self, dt):
        """
        Update the robot's position based on wheel velocities.
        Args:
            dt: Time step in seconds
        """
        # Calculate linear and angular velocity
        v_linear = self.wheel_radius * (self.v_right + self.v_left) / 2.0
        v_angular = self.wheel_radius * (self.v_right - self.v_left) / self.wheel_base
        
        # Update state
        if abs(v_angular) < 1e-6:  # Straight line motion
            self.x += v_linear * cos(self.theta) * dt
            self.y += v_linear * sin(self.theta) * dt
        else:  # Curved motion
            radius = v_linear / v_angular
            self.x += radius * (sin(self.theta + v_angular * dt) - sin(self.theta))
            self.y += radius * (cos(self.theta) - cos(self.theta + v_angular * dt))
            self.theta += v_angular * dt
            
            # Normalize theta to [-pi, pi]
            self.theta = atan2(sin(self.theta), cos(self.theta))
    
    def get_imu_reading(self):
        """
        Get simulated IMU readings (angular velocity and linear acceleration).
        Returns: (angular_velocity, linear_acceleration)
        """
        # Calculate actual values
        v_linear = self.wheel_radius * (self.v_right + self.v_left) / 2.0
        v_angular = self.wheel_radius * (self.v_right - self.v_left) / self.wheel_base
        
        # Add noise
        angular_velocity = v_angular + np.random.normal(0, self.imu_noise_angular_velocity)
        linear_accel = np.random.normal(0, self.imu_noise_linear_accel)  # Simplified, just noise
        
        return angular_velocity, linear_accel
    
    def get_gps_reading(self):
        """
        Get simulated GPS readings.
        Returns: (x, y) with added noise
        """
        gps_x = self.x + np.random.normal(0, self.gps_noise_position)
        gps_y = self.y + np.random.normal(0, self.gps_noise_position)
        
        return gps_x, gps_y
    
    def get_true_state(self):
        """Get the ground truth state."""
        return self.x, self.y, self.theta


class ParticleFilter:
    """
    Particle Filter implementation for differential drive robot localization.
    Each particle represents a possible state [x, y, theta, v, omega].
    """
    def __init__(self, num_particles=300, process_noise=None, gps_noise=1.0, imu_noise=0.01):
        # Number of particles
        self.num_particles = num_particles
        
        # State dimension
        self.n = 5
        
        # Initialize particles randomly
        self.particles = np.zeros((num_particles, self.n))
        
        # Initial particle positions in a small area around the origin
        self.particles[:, 0] = np.random.uniform(-1.0, 1.0, num_particles)  # x
        self.particles[:, 1] = np.random.uniform(-1.0, 1.0, num_particles)  # y
        self.particles[:, 2] = np.random.uniform(-pi, pi, num_particles)    # theta
        self.particles[:, 3] = np.random.uniform(0.0, 0.5, num_particles)   # v
        self.particles[:, 4] = np.random.uniform(-0.2, 0.2, num_particles)  # omega
        
        # Particle weights (initially uniform)
        self.weights = np.ones(num_particles) / num_particles
        
        # Process noise parameters (standard deviations)
        if process_noise is None:
            self.process_noise = np.array([0.05, 0.05, 0.02, 0.1, 0.05])
        else:
            self.process_noise = np.array(process_noise)
        
        # Measurement noise parameters (standard deviations)
        self.gps_noise = gps_noise    # GPS position noise (meters)
        self.imu_noise = imu_noise    # IMU angular velocity noise (rad/s)
        
        # Effective sample size threshold for resampling
        self.resample_threshold = num_particles / 2
    
    def normalize_angle(self, angle):
        """Normalize angle to [-pi, pi]."""
        return atan2(sin(angle), cos(angle))
    
    def predict(self, dt):
        """
        Predict step of the particle filter.
        Args:
            dt: Time step in seconds
        """
        # For each particle, predict next state
        for i in range(self.num_particles):
            # Get current particle state
            x = self.particles[i, 0]
            y = self.particles[i, 1]
            theta = self.particles[i, 2]
            v = self.particles[i, 3]
            omega = self.particles[i, 4]
            
            # Add process noise to control inputs
            v_noisy = v + np.random.normal(0, self.process_noise[3])
            omega_noisy = omega + np.random.normal(0, self.process_noise[4])
            
            # Predict next state based on motion model
            if abs(omega_noisy) < 1e-6:  # Straight line motion
                x_next = x + v_noisy * cos(theta) * dt
                y_next = y + v_noisy * sin(theta) * dt
                theta_next = theta
            else:  # Curved motion
                radius = v_noisy / omega_noisy
                x_next = x + radius * (sin(theta + omega_noisy * dt) - sin(theta))
                y_next = y + radius * (cos(theta) - cos(theta + omega_noisy * dt))
                theta_next = self.normalize_angle(theta + omega_noisy * dt)
            
            # Add process noise to position and orientation
            x_next += np.random.normal(0, self.process_noise[0])
            y_next += np.random.normal(0, self.process_noise[1])
            theta_next += np.random.normal(0, self.process_noise[2])
            theta_next = self.normalize_angle(theta_next)
            
            # Update particle
            self.particles[i, 0] = x_next
            self.particles[i, 1] = y_next
            self.particles[i, 2] = theta_next
            self.particles[i, 3] = v_noisy
            self.particles[i, 4] = omega_noisy
    
    def update_weights_gps(self, gps_x, gps_y):
        """
        Update particle weights based on GPS measurement.
        Args:
            gps_x: GPS x position
            gps_y: GPS y position
        """
        # For each particle, compute likelihood
        for i in range(self.num_particles):
            # Get particle position
            particle_x = self.particles[i, 0]
            particle_y = self.particles[i, 1]
            
            # Compute squared distance error
            dx = gps_x - particle_x
            dy = gps_y - particle_y
            squared_dist = dx*dx + dy*dy
            
            # Compute likelihood using Gaussian model
            likelihood = exp(-squared_dist / (2 * self.gps_noise**2))
            
            # Update weight
            self.weights[i] *= likelihood
        
        # Normalize weights
        if np.sum(self.weights) > 0:
            self.weights /= np.sum(self.weights)
        else:
            # If all weights are zero, reset to uniform
            self.weights = np.ones(self.num_particles) / self.num_particles
    
    def update_weights_imu(self, angular_velocity):
        """
        Update particle weights based on IMU angular velocity measurement.
        Args:
            angular_velocity: Angular velocity from IMU
        """
        # For each particle, compute likelihood
        for i in range(self.num_particles):
            # Get particle angular velocity
            particle_omega = self.particles[i, 4]
            
            # Compute squared error
            squared_error = (angular_velocity - particle_omega)**2
            
            # Compute likelihood using Gaussian model
            likelihood = exp(-squared_error / (2 * self.imu_noise**2))
            
            # Update weight
            self.weights[i] *= likelihood
        
        # Normalize weights
        if np.sum(self.weights) > 0:
            self.weights /= np.sum(self.weights)
        else:
            # If all weights are zero, reset to uniform
            self.weights = np.ones(self.num_particles) / self.num_particles
    
    def resample_if_needed(self):
        """
        Resample particles if the effective sample size is below the threshold.
        """
        # Compute effective sample size
        neff = 1.0 / np.sum(self.weights**2)
        
        # Resample if below threshold
        if neff < self.resample_threshold:
            # Resample with replacement according to weights
            indices = np.random.choice(
                self.num_particles, 
                self.num_particles, 
                replace=True, 
                p=self.weights
            )
            
            # Create new set of particles
            self.particles = self.particles[indices, :]
            
            # Reset weights to uniform
            self.weights = np.ones(self.num_particles) / self.num_particles
    
    def get_state_estimate(self):
        """
        Get the estimated state as the weighted average of particles.
        Returns:
            tuple: (x, y, theta, v, omega)
        """
        # Compute weighted average for position and velocity
        mean_state = np.zeros(self.n)
        for i in range(self.n):
            if i != 2:  # Not theta
                mean_state[i] = np.sum(self.particles[:, i] * self.weights)
        
        # For theta, we need to handle circular mean
        cos_theta = np.sum(np.cos(self.particles[:, 2]) * self.weights)
        sin_theta = np.sum(np.sin(self.particles[:, 2]) * self.weights)
        mean_state[2] = atan2(sin_theta, cos_theta)
        
        return mean_state[0], mean_state[1], mean_state[2], mean_state[3], mean_state[4]
    
    def get_particles(self):
        """Get all particles and their weights for visualization."""
        return self.particles, self.weights


def run_particle_filter_simulation(
    num_particles=300,
    process_noise_xy=0.05,
    process_noise_theta=0.02,
    process_noise_v=0.1,
    process_noise_omega=0.05,
    gps_noise=1.0,
    imu_noise=0.01,
    gps_update_interval=0.5,
    simulation_time=30.0
):
    # Create a robot
    robot = DifferentialDriveRobot()
    robot.gps_noise_position = gps_noise
    robot.imu_noise_angular_velocity = imu_noise
    
    # Create particle filter with specified parameters
    process_noise = [process_noise_xy, process_noise_xy, process_noise_theta, process_noise_v, process_noise_omega]
    pf = ParticleFilter(
        num_particles=num_particles,
        process_noise=process_noise,
        gps_noise=gps_noise,
        imu_noise=imu_noise
    )
    
    # Simulation parameters
    dt = 0.1  # time step
    
    # Storage for plotting
    true_positions = []
    gps_positions = []
    pf_positions = []
    time_points = []
    
    # For saving particles at specific time points
    particles_history = []
    selected_times = [simulation_time * 0.2, simulation_time * 0.5, simulation_time * 0.8]  # Times at which to save particles
    
    # Initial time
    current_time = 0.0
    
    # Main simulation loop
    while current_time < simulation_time:
        # Set wheel velocities (changing over time for interesting trajectory)
        v_left = 1.0 + 0.5 * sin(current_time * 0.2)
        v_right = 1.5 + 0.5 * cos(current_time * 0.3)
        robot.update_wheels_velocity(v_left, v_right)
        
        # Update the robot's true position
        robot.update(dt)
        
        # Get sensor readings
        angular_velocity, _ = robot.get_imu_reading()
        gps_x, gps_y = robot.get_gps_reading()
        
        # Update the Particle Filter
        pf.predict(dt)
        
        # We'll update with IMU at every step
        pf.update_weights_imu(angular_velocity)
        
        # But update with GPS less frequently based on the specified interval
        if current_time % gps_update_interval < dt:
            pf.update_weights_gps(gps_x, gps_y)
            pf.resample_if_needed()
        
        # Store data for plotting
        true_x, true_y, true_theta = robot.get_true_state()
        pf_x, pf_y, pf_theta, pf_v, pf_omega = pf.get_state_estimate()
        
        true_positions.append((true_x, true_y, true_theta))
        gps_positions.append((gps_x, gps_y))
        pf_positions.append((pf_x, pf_y, pf_theta))
        time_points.append(current_time)
        
        # Save particles at selected times
        if any(abs(current_time - t) < dt/2 for t in selected_times):
            particles, weights = pf.get_particles()
            particles_history.append((current_time, particles.copy(), weights.copy()))
        
        # Update time
        current_time += dt
    
    # Convert to numpy arrays for plotting
    true_positions = np.array(true_positions)
    gps_positions = np.array(gps_positions)
    pf_positions = np.array(pf_positions)
    
    # Plot results
    fig = plt.figure(figsize=(15, 10))
    
    # Plot trajectories
    ax1 = plt.subplot(2, 2, 1)
    ax1.plot(true_positions[:, 0], true_positions[:, 1], 'g-', label='True Position')
    ax1.plot(gps_positions[:, 0], gps_positions[:, 1], 'r.', markersize=3, label='GPS Measurements')
    ax1.plot(pf_positions[:, 0], pf_positions[:, 1], 'b-', label='PF Estimate')
    ax1.set_xlabel('X Position (m)')
    ax1.set_ylabel('Y Position (m)')
    ax1.legend()
    ax1.set_title('Robot Trajectory')
    ax1.grid(True)
    
    # Plot position error over time
    ax2 = plt.subplot(2, 2, 2)
    position_error = np.sqrt((pf_positions[:, 0] - true_positions[:, 0])**2 + 
                            (pf_positions[:, 1] - true_positions[:, 1])**2)
    ax2.plot(time_points, position_error)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Position Error (m)')
    ax2.set_title('Position Estimation Error')
    ax2.grid(True)
    
    # Plot orientation error over time
    ax3 = plt.subplot(2, 2, 3)
    # Calculate angular difference, handling circular nature
    orientation_error = np.array([min(abs(pf_positions[i, 2] - true_positions[i, 2]), 
                                     2*pi - abs(pf_positions[i, 2] - true_positions[i, 2])) 
                                 for i in range(len(time_points))])
    ax3.plot(time_points, orientation_error)
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Orientation Error (rad)')
    ax3.set_title('Orientation Estimation Error')
    ax3.grid(True)
    
    # Plot effective sample size over time
    ax4 = plt.subplot(2, 2, 4)
    # We don't have this data from the simulation, so we'll just show a placeholder
    ax4.text(0.5, 0.5, 'Parameter Summary:\n' +
             f'Particles: {num_particles}\n' +
             f'Process Noise XY: {process_noise_xy}\n' +
             f'Process Noise θ: {process_noise_theta}\n' +
             f'GPS Noise: {gps_noise}\n' +
             f'IMU Noise: {imu_noise}\n' +
             f'GPS Update: every {gps_update_interval}s',
             horizontalalignment='center',
             verticalalignment='center',
             transform=ax4.transAxes)
    ax4.set_title('Simulation Parameters')
    ax4.set_xticks([])
    ax4.set_yticks([])
    
    plt.tight_layout()
    
    # Create a new figure for particle distribution visualizations
    if particles_history:
        fig2 = plt.figure(figsize=(15, 5))
        
        # Plot particle distributions at specific times
        for i, (t, particles, weights) in enumerate(particles_history):
            ax = plt.subplot(1, len(particles_history), i + 1)
            
            # Plot true position
            idx = np.argmin(np.abs(np.array(time_points) - t))
            true_x, true_y = true_positions[idx, 0], true_positions[idx, 1]
            true_theta = true_positions[idx, 2]
            
            # Plot particles with transparency based on weight
            max_weight = np.max(weights)
            for j in range(len(particles)):
                alpha = min(1.0, weights[j] / max_weight * 10)
                ax.scatter(particles[j, 0], particles[j, 1], color='b', alpha=alpha, s=10)
            
            # Plot arrows for a subset of particles to show orientation
            subsample = max(1, num_particles // 30)  # Plot orientation for roughly 30 particles
            for j in range(0, len(particles), subsample):
                ax.arrow(particles[j, 0], particles[j, 1], 
                        0.2 * cos(particles[j, 2]), 0.2 * sin(particles[j, 2]),
                        head_width=0.1, head_length=0.2, fc='b', ec='b', alpha=0.5)
            
            # Plot true position and orientation
            ax.scatter(true_x, true_y, color='g', s=100, marker='*')
            ax.arrow(true_x, true_y, 
                    0.3 * cos(true_theta), 0.3 * sin(true_theta),
                    head_width=0.15, head_length=0.3, fc='g', ec='g')
            
            ax.set_title(f'Particle Distribution at t={t:.1f}s')
            ax.set_xlabel('X Position (m)')
            ax.set_ylabel('Y Position (m)')
            ax.grid(True)
        
        plt.tight_layout()
    
    plt.show()
    
    # Return error metrics for comparison
    mean_pos_error = np.mean(position_error)
    max_pos_error = np.max(position_error)
    mean_orient_error = np.mean(orientation_error)
    
    return {
        'mean_position_error': mean_pos_error,
        'max_position_error': max_pos_error,
        'mean_orientation_error': mean_orient_error
    }


def interactive_particle_filter_demo():
    """
    Create interactive widgets to demonstrate the particle filter with adjustable parameters.
    """
    # Create widgets for parameter adjustment
    particles_slider = widgets.IntSlider(
        value=300,
        min=50,
        max=1000,
        step=50,
        description='Particles:',
        disabled=False,
        continuous_update=False
    )
    
    process_noise_xy_slider = widgets.FloatSlider(
        value=0.05,
        min=0.01,
        max=0.2,
        step=0.01,
        description='Process Noise XY:',
        disabled=False,
        continuous_update=False
    )
    
    process_noise_theta_slider = widgets.FloatSlider(
        value=0.02,
        min=0.005,
        max=0.1,
        step=0.005,
        description='Process Noise θ:',
        disabled=False,
        continuous_update=False
    )
    
    process_noise_v_slider = widgets.FloatSlider(
        value=0.1,
        min=0.02,
        max=0.3,
        step=0.02,
        description='Process Noise v:',
        disabled=False,
        continuous_update=False
    )
    
    process_noise_omega_slider = widgets.FloatSlider(
        value=0.05,
        min=0.01,
        max=0.2,
        step=0.01,
        description='Process Noise ω:',
        disabled=False,
        continuous_update=False
    )
    
    gps_noise_slider = widgets.FloatSlider(
        value=1.0,
        min=0.1,
        max=3.0,
        step=0.1,
        description='GPS Noise:',
        disabled=False,
        continuous_update=False
    )
    
    imu_noise_slider = widgets.FloatSlider(
        value=0.01,
        min=0.001,
        max=0.05,
        step=0.001,
        description='IMU Noise:',
        disabled=False,
        continuous_update=False
    )
    
    gps_update_slider = widgets.FloatSlider(
        value=0.5,
        min=0.1,
        max=2.0,
        step=0.1,
        description='GPS Update (s):',
        disabled=False,
        continuous_update=False
    )
    
    sim_time_slider = widgets.FloatSlider(
        value=15.0,
        min=5.0,
        max=30.0,
        step=5.0,
        description='Sim Time (s):',
        disabled=False,
        continuous_update=False
    )
    
    run_button = widgets.Button(
        description='Run Simulation',
        disabled=False,
        button_style='success',
        tooltip='Click to run the simulation with the current parameters',
        icon='play'
    )
    
    output = widgets.Output()
    
    # Define the callback function for the run button
    def on_run_button_clicked(b):
        with output:
            output.clear_output()
            print("Running simulation...")
            metrics = run_particle_filter_simulation(
                num_particles=particles_slider.value,
                process_noise_xy=process_noise_xy_slider.value,
                process_noise_theta=process_noise_theta_slider.value,
                process_noise_v=process_noise_v_slider.value,
                process_noise_omega=process_noise_omega_slider.value,
                gps_noise=gps_noise_slider.value,
                imu_noise=imu_noise_slider.value,
                gps_update_interval=gps_update_slider.value,
                simulation_time=sim_time_slider.value
            )
            print("\nPerformance Metrics:")
            print(f"Mean Position Error: {metrics['mean_position_error']:.4f} m")
            print(f"Max Position Error: {metrics['max_position_error']:.4f} m")
            print(f"Mean Orientation Error: {metrics['mean_orientation_error']:.4f} rad")
    
    # Attach the callback to the button
    run_button.on_click(on_run_button_clicked)
    
    # Create layout of widgets
    left_panel = widgets.VBox([
        particles_slider,
        process_noise_xy_slider,
        process_noise_theta_slider,
        process_noise_v_slider,
        process_noise_omega_slider
    ])
    
    right_panel = widgets.VBox([
        gps_noise_slider,
        imu_noise_slider,
        gps_update_slider,
        sim_time_slider,
        run_button
    ])
    
    # Combine widgets and output
    app = widgets.VBox([
        widgets.HTML("<h2>Particle Filter for Differential Drive Robot</h2>"),
        widgets.HTML("<p>Adjust parameters and click 'Run Simulation' to see the results.</p>"),
        widgets.HBox([left_panel, right_panel]),
        output
    ])
    
    # Display the interactive application
    display(app)
    
    # Return the app in case it's needed
    return app


# Display the interactive demo
interactive_particle_filter_demo()